In [2]:
import pandas as pd
import numpy as np
import itertools
import functools
from nl_03_filter_model_score import filter_split_model_score
import importlib.util

This notebook, nl_03_filter_model_score_nb.ipynb, serves as an interactive interface for 
filtering, splitting, modeling, and scoring neutral loss METASPACE data.

Cells with interactive parameters are set below:

In [ ]:
# df4 = pd.evalute('df1 + df2 + df3') # Much faster for big DF!
# Basic math and logic mostly...

In [ ]:
# Load ML DF, then load bits or mord, use expressions below to find
# idx hitting filtering criteria.  Filter ML DF then filter bits or 
# mord based on same indexes returned

idx = df.query('logic').index.tolist()
idx = df[df['col_A' > 100]].index.tolist()

In [ ]:
#Try each, check for memory and poor column type usage
mord_df = pd.read_pickle('temp_mord.pickle')
bits_df = pd.read_pickle('temp_bits.pickle')
join_df = pd.read_pickle('/Users/dis/PycharmProjects/neutral_loss/all_public_output_02.pickle')

In [18]:
# Filtering:
target = ['H2O'] # ['H2O']
polarity = [1] # {1:positive, -1:negative] 
fdrs = [0.05] # [0.2, 0.1, 0.05]
colocalizations = [0.75] # [0, 0.5, 0.75]

# Outputs (y), and weights (w):
global_ys = ['n_loss_only_H2O', 'n_loss_wparent_H2O']
global_y = [global_ys[1]]
global_w = ['weight']

# Class of model:
direct_model_on = False
ml_model_on = True
deepchem_model_on = False

# Split.  If true, any formula only appears in train/test/validate.
single_fold_group = True

direct_df = pd.DataFrame()
ml_df = pd.DataFrame()
dc_df = pd.DataFrame()

# Specific inputs (X) for each model:
if direct_model_on is True:
    model = ['']
    Xs = ['trues', 'falses', 'rando', 'H2O_Present', 'n_loss_wparent_H2O']
    X =  [Xs[3]]
    y = global_y
    w = global_w
    direct_models = list(itertools.product(*[target, polarity, fdrs, colocalizations,
                                             ['direct'], model, X, y, w]))
    
    direct_df = pd.DataFrame(direct_models, columns=['target', 'polarity', 'fdr', 'coloc',
                                       'mclass', 'model', 'X', 'y', 'w'])
    
if ml_model_on is True:
    models = ['random_forest', 'XGBoost', 'ThunderSVM']
    model = [models[1]]
    Xs = ['bits', 'mord_norm'] 
    X = [Xs[0]]
    y = global_y
    w = global_w
    ml_params = []
    ml_models = list(itertools.product(*[target, polarity, fdrs, colocalizations,
                                         ['ml'], model, X, y, w]))
    ml_df = pd.DataFrame(ml_models, columns=['target', 'polarity', 'fdr', 'coloc',
                                       'mclass', 'model', 'X', 'y', 'w']) 

if deepchem_model_on is True:
    models = [('GraphConvModel', 'GraphConv'), 
              ('WeaveModel', 'Weave'), 
              ('MPNNModel','Weave') ] 
    model = [models[0]]    
    Xs = ['Molecule', 'Smiles']
    X = [Xs[0]]          
    y = global_y
    w = global_w
    dc_params = []
    dc_models = list(itertools.product(*[target, polarity, fdrs, colocalizations,
                                         ['dc'], model, X, y, w]))
    dc_df = pd.DataFrame(dc_models, columns=['target', 'polarity', 'fdr', 'coloc',
                                       'mclass', 'model', 'X', 'y', 'w'])
    
filter_param_df = pd.concat([direct_df, ml_df, dc_df]).reset_index(drop=True)
#model_param = [ml_params, dc_params]
filter_param_df

,target,polarity,fdr,coloc,mclass,model,X,y,w
0,H2O,positive,0.05,0.75,ml,XGBoost,bits,n_loss_wparent_H2O,weight


In [19]:
result = filter_split_model_score(filter_param_df, join_df, single_fold_group, 
                                  target) #model_params

start 0


KeyError: "['bits'] not in index"

In [427]:
result.round(3)

In [428]:
name = 'this_file_name'
result.to_pickle(name)

,target,polarity,fdr,coloc,mclass,model,X,y,w,sens,spec,f1,test_n,test_true,tp,tn,fp,fn,acc_train,acc_test
0,H2O,positive,0.05,0.75,ml,random_forest,bits,n_loss_wparent_H2O,weight,0.485,0.629,0.197,140476,53618,6622,79834,46996,7024,0.768,0.768
1,H2O,positive,0.05,0.75,ml,random_forest,mord_norm,n_loss_wparent_H2O,weight,0.323,0.896,0.282,140476,17587,4403,113646,13184,9243,0.826,0.826
2,H2O,positive,0.05,0.75,ml,random_forest,fp_feats,n_loss_wparent_H2O,weight,0.335,0.869,0.262,140476,21230,4570,110170,16660,9076,0.826,0.826


In [ ]:
# Grid search?  Use a row from filter pslit.